### loading modules and data

In [0]:
########################################################

# If running with Google Colab

from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# # If using colab
from google.colab import files
files.upload()
# # select the 3 .py files (models, utils, data_utils)

In [0]:
########################################################

# If running with Google Colab
# Create a folder "C247" and then store the project datasets within that folder
# Check that your datasets are setup correctly

!ls "/content/gdrive/My Drive/C247" # File path

In [0]:
from models import *
from utils import *
from data_utils import *

In [0]:
# X_test = np.load("X_test.npy")
# y_test = np.load("y_test.npy")
# person_train_valid = np.load("person_train_valid.npy")
# X_train_valid = np.load("X_train_valid.npy")
# y_train_valid = np.load("y_train_valid.npy")
# person_test = np.load("person_test.npy")

# Change if your directory is different

# dataset_path = './data/' # Yiming Path
dataset_path = "/content/gdrive/My Drive/C247/" 

X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


### CWT 0.1 & 0.4

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0 & 0.4

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
bottom_scale = 0
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0.1 & 0.5

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.5
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0.2 & 0.5

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.5
bottom_scale = 0.2
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0 & 0.3

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.3
bottom_scale = 0
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0.1 & 0.3

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.3
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 190
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0.2 & 0.4

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.4
bottom_scale = 0.2
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0.2 & 0.3

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.3
bottom_scale = 0.2
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

### CWT 0.1 & 0.2

In [0]:
# Uncomment for window
# train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)

# Uncomment for CWT -> Window
num_levels = 5
top_scale = 0.2
bottom_scale = 0.1
X_cwt, y_cwt, p_cwt = cwt_data2(X_train_valid, y_train_valid, person_train_valid, num_levels, bottom_scale = bottom_scale, top_scale=top_scale)
train_fold, val_fold = Train_Val_Data(X_cwt, y_cwt)
X_cwt[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 50
k = 0
dropout = 0.75

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.0001]
X_train, y_train, p_train = X_cwt[train_fold[k]], y_cwt[train_fold[k]], p_cwt[train_fold[k]]
X_val, y_val, p_val = X_cwt[val_fold[k]], y_cwt[val_fold[k]], p_cwt[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


#### Training and Testing

In [0]:
# Uncomment if using cwt
X_cwt_test, y_cwt_test, person_cwt_test = cwt_data2(X_test, y_test, person_test, num_levels, top_scale=top_scale)

TestRNN(model, X_cwt_test, y_cwt_test, person_cwt_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

#ICA 15

In [0]:
X_train_ICA = np.load(dataset_path + "X_train_ICA15.npy")
X_test_ICA  = np.load(dataset_path + "X_test_ICA15.npy")

In [0]:
train_fold, val_fold = Train_Val_Data(X_train_ICA, y_train_valid)
X_train_ICA[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 500
stride = 200 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 100
k = 0
dropout = 0.25

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.001]
X_train, y_train, p_train = X_train_ICA[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_ICA[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', input_size=X_train.shape[1], dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


In [0]:
# Uncomment if using cwt
TestRNN(model, X_test_ICA, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

#ICA 5

In [0]:
X_train_ICA = np.load(dataset_path + "X_train_ICA5.npy")
X_test_ICA  = np.load(dataset_path + "X_test_ICA5.npy")

In [0]:
train_fold, val_fold = Train_Val_Data(X_train_ICA, y_train_valid)
X_train_ICA[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 500
stride = 200 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 100
k = 0
dropout = 0.25

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.001]
X_train, y_train, p_train = X_train_ICA[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_ICA[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', input_size=X_train.shape[1], dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


In [0]:
# Uncomment if using cwt
TestRNN(model, X_test_ICA, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

#ICA 10 BP filtered (30,50hz)

In [0]:
X_train_ICA = np.load(dataset_path + "X_train_ICA10.npy")
X_test_ICA  = np.load(dataset_path + "X_test_ICA10.npy")

In [0]:
train_fold, val_fold = Train_Val_Data(X_train_ICA, y_train_valid)
X_train_ICA[train_fold[0]].shape

In [0]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
stride = 50 # Make 20 normally
vote_num = 50 # Make 20 normally
num_folds = 1
num_epochs = 100
k = 0
dropout = 0.25

#drops = [0.5, 0.75]
learn_rates = [0.001, 0.0005, 0.0001]
lrn_rt = [0.001]
X_train, y_train, p_train = X_train_ICA[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
X_val, y_val, p_val = X_train_ICA[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size, window_stride=stride)
if aug_type != 'window':
    X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)

for lrn_rt in lrn_rt:
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='GRU', input_size=X_train.shape[1], dropout=dropout, weight_decay=0, lr=lrn_rt)
    print ('lrn_rt {}'.format(lrn_rt))
    
    # Uncomment for just windowing
    # X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    # X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]

    # Uncomment for cwt
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=num_epochs, aug_type=aug_type, window_size=window_size, vote_num=vote_num)
